In [1]:
import numpy as np
import netCDF4 as nc
import scipy as sc

In [2]:
!ls

bathy_files
bathy_meter.nc
C:\nppdf32Log\debuglog.txt
coordinates.nc
Dealing_with_FRS_initial_T_and_S_remaking_temp_sal_BC.ipynb
Detiding.ipynb
Fill_up_holes_in_IC_file_and_BC_file.ipynb
grid_alignment.py
Grid_transformation.ipynb
input10
input11
input12
input13
input14
input15
input16
input17
input18
input19
input20
input21
input22
input23
input24
input25
input26
input27
input28
input29
input30
input31
input32
input33
input34
input35
input36
input37
input38
input39
input4
input40
input41
input42
input43
input44
input45
input46
input47
input48
input49
input5
input50
input51
input52
input53
input54
input55
input56
input57
input58
input59
input6
input60
input61
input62
input63
input64
input65
input66
input67
input68
input69
input7
input70
input71
input72
input73
input74
input75
input76
input77
input78
input79
input8
input80
input81
input82
input83
input84
input85
input86
input87
input88
input89
input9
input90
input91
input92
input93
input94
input95
input96
input97
input98
input99
JP_Grid

In [3]:
u_3d_left = np.load('U_3D_wcvi_left.npy'); v_3d_left_oneless = np.load('V_3D_wcvi_left.npy');

ssh_left = np.load('ssh_leftbc.npy'); 

votemper_left = np.load('votemper_leftbc.npy'); vosaline_left = np.load('vosaline_leftbc.npy');

##### Seeing the shapes of the arrays of U and V (both 2d and 3D) we see that we have one grid point less in V in the y grid)... so we make that value equal to the immediately next grid point.

### Point to note for future: Extrapolate the data to the missing point instead of making them equal (else make once cell bigger and simply trim it off to match our shape needed)

In [4]:
u_2d_left = np.mean(u_3d_left, axis= 1); v_2d_left_oneless = np.mean(v_3d_left_oneless, axis=1);

v_2d_left = np.empty(u_2d_left.shape); 
v_2d_left[:,:-1,:] = v_2d_left_oneless;
v_2d_left[:,-1,:] = v_2d_left[:,-2,:];

v_3d_left = np.empty(u_3d_left.shape);
v_3d_left[:,:,:-1,:] = v_3d_left_oneless;
v_3d_left[:,:,-1,:] = v_3d_left[:,:,-2,:];


In [5]:
votemper_left.shape

(43, 50, 100, 4)

In [6]:
v_2d_left.shape

(43, 100, 4)

In [7]:
u_2d_left.shape

(43, 100, 4)

### Now let us swap the axes to match the NEMO wants the input files 
(refer to the ssh files in /ocean/mdunphy/MEOPAR/NEMO-forcing/open_boundaries/west/ssh)

In [8]:
print (ssh_left.shape, np.swapaxes(ssh_left, 1, 2).shape)

(43, 100, 4) (43, 4, 100)


In [9]:
u_2d_left.shape

(43, 100, 4)

#### Now we will write these down in a file to make the leftmost boundary work with unstructured open boundaries

In [10]:
bdy_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/2d_west_m04.nc', 'w', zlib=True);

bdy_file.createDimension('xbT', u_3d_left.shape[2]);
bdy_file.createDimension('yb', u_3d_left.shape[3]);
bdy_file.createDimension('time_counter', None);

xbT = bdy_file.createVariable('xbT', 'int32', ('xbT',), zlib=True);
xbT.units = 'indices';
xbT.longname = 'indices along left boundary';

yb = bdy_file.createVariable('yb', 'int32', ('yb',), zlib=True);
yb.units = 'indices';
yb.longname = 'a strip of indices across all of left boundary';

time_counter = bdy_file.createVariable('time_counter', 'int32', ('time_counter',), zlib=True);
time_counter.units = 's';
time_counter.longname = 'time';

sossheig = bdy_file.createVariable('sossheig', 'float32', ('time_counter', 'yb', 'xbT'), zlib=True);
vobtcrtx = bdy_file.createVariable('vobtcrtx', 'float32', ('time_counter', 'yb', 'xbT'), zlib=True);
vobtcrty = bdy_file.createVariable('vobtcrty', 'float32', ('time_counter', 'yb', 'xbT'), zlib=True);


sossheig[:] = np.swapaxes(ssh_left, 1, 2)[:];
sossheig[:] = sossheig[:];  ## done so that NEMO can read it the other way around
vobtcrtx[:] = np.swapaxes(u_2d_left, 1, 2)[:];
vobtcrtx[:] = vobtcrtx[:];
vobtcrty[:] = np.swapaxes(v_2d_left, 1, 2)[:];
vobtcrty[:] = vobtcrty[:];

bdy_file.close()


In [11]:
v_2d_left.shape

(43, 100, 4)

In [12]:
print(u_3d_left.shape , np.swapaxes(u_3d_left, 3, 2).shape)

(43, 50, 100, 4) (43, 50, 4, 100)


In [13]:
v_3d_left.shape

(43, 50, 100, 4)

In [14]:
votemper_left.shape

(43, 50, 100, 4)

In [15]:
!ncdump -h /ocean/ssahu/CANYONS/bdy_files/2d_left.nc

*** ncdump: ncopen failed on /ocean/ssahu/CANYONS/bdy_files/2d_left.nc


#### NEMO has a provision to give the total velcities as input by specifying 

     ln_full_vel = .false.		        ! =T => full velocities in 3D boundary conditions
                                        ! =F => baroclinic velocities in 3D boundary conditions
                                        
         So after getting the barotropic (2D) it is not necessary to subtract the depth averaged velocities from their 3D counterparts to get the barocloinic velocities.

In [16]:
bdy_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/3d_west_m04.nc', 'w', zlib=True);


bdy_file.createDimension('xbT', u_3d_left.shape[2]);
bdy_file.createDimension('yb', u_3d_left.shape[3]);
bdy_file.createDimension('deptht', u_3d_left.shape[1]);
bdy_file.createDimension('time_counter', None);


xbT = bdy_file.createVariable('xbT', 'int32', ('xbT',), zlib=True);
xbT.units = 'indices';
xbT.longname = 'indices along left boundary';

yb = bdy_file.createVariable('yb', 'int32', ('yb',), zlib=True);
yb.units = 'indices';
yb.longname = 'a strip of indices across all of left boundary';

deptht = bdy_file.createVariable('deptht', 'int32', ('deptht',), zlib=True);
deptht.units = 'm';
deptht.longname = 'Vertical T Levels';

time_counter = bdy_file.createVariable('time_counter', 'int32', ('time_counter',), zlib=True);
time_counter.units = 's';
time_counter.longname = 'time';

vozocrtx = bdy_file.createVariable('vozocrtx', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);
vomecrty = bdy_file.createVariable('vomecrty', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);
votemper = bdy_file.createVariable('votemper', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);
vosaline = bdy_file.createVariable('vosaline', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);

vozocrtx[:] = np.swapaxes(u_3d_left, 3, 2)[:];
vomecrty[:] = np.swapaxes(v_3d_left, 3, 2)[:];
votemper[:] = np.swapaxes(votemper_left, 3, 2)[:];
vosaline[:] = np.swapaxes(vosaline_left, 3, 2)[:];

bdy_file.close()




#### Make the east boundary

In [17]:
u_3d_right = np.load('U_3D_wcvi_right.npy'); v_3d_right_oneless = np.load('V_3D_wcvi_right.npy');

ssh_right = np.load('ssh_rightbc.npy'); 

votemper_right = np.load('votemper_rightbc.npy'); vosaline_right = np.load('vosaline_rightbc.npy');

u_2d_right = np.mean(u_3d_right, axis= 1); v_2d_right_oneless = np.mean(v_3d_right_oneless, axis=1);

v_2d_right = np.empty(u_2d_right.shape); 
v_2d_right[:,:-1,:] = v_2d_right_oneless;
v_2d_right[:,-1,:] = v_2d_right[:,-2,:];

v_3d_right = np.empty(u_3d_right.shape);
v_3d_right[:,:,:-1,:] = v_3d_right_oneless;
v_3d_right[:,:,-1,:] = v_3d_right[:,:,-2,:];

In [18]:
bdy_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/2d_right_m04.nc', 'w', zlib=True);

bdy_file.createDimension('xbT', u_3d_right.shape[2]);
bdy_file.createDimension('yb', u_3d_right.shape[3]);
bdy_file.createDimension('time_counter', None);

xbT = bdy_file.createVariable('xbT', 'int32', ('xbT',), zlib=True);
xbT.units = 'indices';
xbT.longname = 'indices along right boundary';

yb = bdy_file.createVariable('yb', 'int32', ('yb',), zlib=True);
yb.units = 'indices';
yb.longname = 'a strip of indices across all of right boundary';

time_counter = bdy_file.createVariable('time_counter', 'int32', ('time_counter',), zlib=True);
time_counter.units = 's';
time_counter.longname = 'time';

sossheig = bdy_file.createVariable('sossheig', 'float32', ('time_counter', 'yb', 'xbT'), zlib=True);
vobtcrtx = bdy_file.createVariable('vobtcrtx', 'float32', ('time_counter', 'yb', 'xbT'), zlib=True);
vobtcrty = bdy_file.createVariable('vobtcrty', 'float32', ('time_counter', 'yb', 'xbT'), zlib=True);


sossheig[:] = np.swapaxes(ssh_right, 1, 2)[:];
vobtcrtx[:] = np.swapaxes(u_2d_right, 1, 2)[:];
vobtcrty[:] = np.swapaxes(v_2d_right, 1, 2)[:];
sossheig[:] = sossheig[:];
vobtcrtx[:] = vobtcrtx[:];
vobtcrty[:] = vobtcrty[:];

bdy_file.close()

In [19]:
bdy_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/3d_right_m04.nc', 'w', zlib=True);


bdy_file.createDimension('xbT', u_3d_right.shape[2]);
bdy_file.createDimension('yb', u_3d_right.shape[3]);
bdy_file.createDimension('deptht', u_3d_right.shape[1]);
bdy_file.createDimension('time_counter', None);


xbT = bdy_file.createVariable('xbT', 'int32', ('xbT',), zlib=True);
xbT.units = 'indices';
xbT.longname = 'indices along right boundary';

yb = bdy_file.createVariable('yb', 'int32', ('yb',), zlib=True);
yb.units = 'indices';
yb.longname = 'a strip of indices across all of right boundary';

deptht = bdy_file.createVariable('deptht', 'int32', ('deptht',), zlib=True);
deptht.units = 'm';
deptht.longname = 'Vertical T Levels';

time_counter = bdy_file.createVariable('time_counter', 'int32', ('time_counter',), zlib=True);
time_counter.units = 's';
time_counter.longname = 'time';

vozocrtx = bdy_file.createVariable('vozocrtx', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);
vomecrty = bdy_file.createVariable('vomecrty', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);
votemper = bdy_file.createVariable('votemper', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);
vosaline = bdy_file.createVariable('vosaline', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);

vozocrtx[:] = np.swapaxes(u_3d_right, 3, 2)[:];
vomecrty[:] = np.swapaxes(v_3d_right, 3, 2)[:];
votemper[:] = np.swapaxes(votemper_right, 3, 2)[:];
vosaline[:] = np.swapaxes(vosaline_right, 3, 2)[:];

bdy_file.close()




#### Making the north boundary

In [20]:
u_3d_top_twoless = np.load('U_3D_wcvi_top.npy'); v_3d_top_oneless = np.load('V_3D_wcvi_top.npy');

ssh_top = np.load('ssh_topbc.npy'); 

votemper_top = np.load('votemper_topbc.npy'); vosaline_top = np.load('vosaline_topbc.npy');

u_2d_top_twoless = np.mean(u_3d_top_twoless, axis= 1); v_2d_top_oneless = np.mean(v_3d_top_oneless, axis=1);

u_2d_top = np.empty((43,4,70)); 
u_2d_top[:,:,:-2] = u_2d_top_twoless;
u_2d_top[:,:,-1] = u_2d_top[:,:,-3];
u_2d_top[:,:,-2] = u_2d_top[:,:,-3];

u_3d_top = np.empty((43,50,4,70)); 
u_3d_top[:,:,:,:-2] = u_3d_top_twoless;
u_3d_top[:,:,:,-1] = u_3d_top[:,:,:,-3];
u_3d_top[:,:,:,-2] = u_3d_top[:,:,:,-3];


v_2d_top = np.empty((43,4,70)); 
v_2d_top[:,:,:-1] = v_2d_top_oneless;
v_2d_top[:,:,-1] = v_2d_top[:,:,-2];


v_3d_top = np.empty((43,50,4,70)); 
v_3d_top[:,:,:,:-1] = v_3d_top_oneless;
v_3d_top[:,:,:,-1] = v_3d_top[:,:,:,-2];

In [21]:
bdy_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/2d_north_m04.nc', 'w', zlib=True);

bdy_file.createDimension('xbT', u_3d_top.shape[3]);
bdy_file.createDimension('yb', u_3d_top.shape[2]);
bdy_file.createDimension('time_counter', None);

xbT = bdy_file.createVariable('xbT', 'int32', ('xbT',), zlib=True);
xbT.units = 'indices';
xbT.longname = 'x indices along north boundary';

yb = bdy_file.createVariable('yb', 'int32', ('yb',), zlib=True);
yb.units = 'indices';
yb.longname = 'a strip of y indices across all of north boundary';

time_counter = bdy_file.createVariable('time_counter', 'int32', ('time_counter',), zlib=True);
time_counter.units = 's';
time_counter.longname = 'time';

sossheig = bdy_file.createVariable('sossheig', 'float32', ('time_counter', 'yb', 'xbT'), zlib=True);
vobtcrtx = bdy_file.createVariable('vobtcrtx', 'float32', ('time_counter', 'yb', 'xbT'), zlib=True);
vobtcrty = bdy_file.createVariable('vobtcrty', 'float32', ('time_counter', 'yb', 'xbT'), zlib=True);


sossheig[:] = ssh_top[:];
vobtcrtx[:] = u_2d_top[:];
vobtcrty[:] = v_2d_top[:];

sossheig[:] = sossheig[:];
vobtcrtx[:] = vobtcrtx[:];
vobtcrty[:] = vobtcrty[:];

bdy_file.close()


In [22]:
bdy_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/3d_north_m04.nc', 'w', zlib=True);


bdy_file.createDimension('xbT', u_3d_top.shape[3]);
bdy_file.createDimension('yb', u_3d_top.shape[2]);
bdy_file.createDimension('deptht', u_3d_top.shape[1]);
bdy_file.createDimension('time_counter', None);


xbT = bdy_file.createVariable('xbT', 'int32', ('xbT',), zlib=True);
xbT.units = 'indices';
xbT.longname = 'x indices along north boundary';

yb = bdy_file.createVariable('yb', 'int32', ('yb',), zlib=True);
yb.units = 'indices';
yb.longname = 'a strip of y indices across all of north boundary';

deptht = bdy_file.createVariable('deptht', 'int32', ('deptht',), zlib=True);
deptht.units = 'm';
deptht.longname = 'Vertical T Levels';

time_counter = bdy_file.createVariable('time_counter', 'int32', ('time_counter',), zlib=True);
time_counter.units = 's';
time_counter.longname = 'time';

vozocrtx = bdy_file.createVariable('vozocrtx', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);
vomecrty = bdy_file.createVariable('vomecrty', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);
votemper = bdy_file.createVariable('votemper', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);
vosaline = bdy_file.createVariable('vosaline', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);

vozocrtx[:] = u_3d_top[:];
vomecrty[:] = v_3d_top[:];
votemper[:] = votemper_top[:];
vosaline[:] = vosaline_top[:];

bdy_file.close()


In [23]:
u_2d_top.shape

(43, 4, 70)

In [24]:
v_3d_top.shape

(43, 50, 4, 70)

In [25]:
u_3d_top.shape

(43, 50, 4, 70)

In [26]:
ssh_top.shape

(43, 4, 70)

##### Making the bottom boundary

In [27]:
u_3d_bottom_twoless = np.load('U_3D_wcvi_bottom.npy'); v_3d_bottom_oneless = np.load('V_3D_wcvi_bottom.npy');

ssh_bottom = np.load('ssh_bottombc.npy'); 

votemper_bottom = np.load('votemper_bottombc.npy'); vosaline_bottom = np.load('vosaline_bottombc.npy');


u_2d_bottom_twoless = np.mean(u_3d_bottom_twoless, axis= 1); v_2d_bottom_oneless = np.mean(v_3d_bottom_oneless, axis=1);

u_2d_bottom = np.empty((43,4,70)); 
u_2d_bottom[:,:,:-2] = u_2d_bottom_twoless;
u_2d_bottom[:,:,-1] = u_2d_bottom[:,:,-3];
u_2d_bottom[:,:,-2] = u_2d_bottom[:,:,-3];

u_3d_bottom = np.empty((43,50,4,70)); 
u_3d_bottom[:,:,:,:-2] = u_3d_bottom_twoless;
u_3d_bottom[:,:,:,-1] = u_3d_bottom[:,:,:,-3];
u_3d_bottom[:,:,:,-2] = u_3d_bottom[:,:,:,-3];


v_2d_bottom = np.empty((43,4,70)); 
v_2d_bottom[:,:,:-1] = v_2d_bottom_oneless;
v_2d_bottom[:,:,-1] = v_2d_bottom[:,:,-2];


v_3d_bottom = np.empty((43,50,4,70)); 
v_3d_bottom[:,:,:,:-1] = v_3d_bottom_oneless;
v_3d_bottom[:,:,:,-1] = v_3d_bottom[:,:,:,-2];


In [28]:
bdy_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/2d_south_m04.nc', 'w', zlib=True);

bdy_file.createDimension('xbT', u_3d_bottom.shape[3]);
bdy_file.createDimension('yb', u_3d_bottom.shape[2]);
bdy_file.createDimension('time_counter', None);

xbT = bdy_file.createVariable('xbT', 'int32', ('xbT',), zlib=True);
xbT.units = 'indices';
xbT.longname = 'x indices along south boundary';

yb = bdy_file.createVariable('yb', 'int32', ('yb',), zlib=True);
yb.units = 'indices';
yb.longname = 'a strip of y indices across all of south boundary';

time_counter = bdy_file.createVariable('time_counter', 'int32', ('time_counter',), zlib=True);
time_counter.units = 's';
time_counter.longname = 'time';

sossheig = bdy_file.createVariable('sossheig', 'float32', ('time_counter', 'yb', 'xbT'), zlib=True);
vobtcrtx = bdy_file.createVariable('vobtcrtx', 'float32', ('time_counter', 'yb', 'xbT'), zlib=True);
vobtcrty = bdy_file.createVariable('vobtcrty', 'float32', ('time_counter', 'yb', 'xbT'), zlib=True);


sossheig[:] = ssh_bottom[:];
vobtcrtx[:] = u_2d_bottom[:];
vobtcrty[:] = v_2d_bottom[:];

sossheig[:] = sossheig[:];
vobtcrtx[:] = vobtcrtx[:];
vobtcrty[:] = vobtcrty[:];

bdy_file.close()

In [29]:
bdy_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/3d_south_m04.nc', 'w', zlib=True);


bdy_file.createDimension('xbT', u_3d_bottom.shape[3]);
bdy_file.createDimension('yb', u_3d_bottom.shape[2]);
bdy_file.createDimension('deptht', u_3d_bottom.shape[1]);
bdy_file.createDimension('time_counter', None);


xbT = bdy_file.createVariable('xbT', 'int32', ('xbT',), zlib=True);
xbT.units = 'indices';
xbT.longname = 'x indices along south boundary';

yb = bdy_file.createVariable('yb', 'int32', ('yb',), zlib=True);
yb.units = 'indices';
yb.longname = 'a strip of y indices across all of south boundary';

deptht = bdy_file.createVariable('deptht', 'int32', ('deptht',), zlib=True);
deptht.units = 'm';
deptht.longname = 'Vertical T Levels';

time_counter = bdy_file.createVariable('time_counter', 'int32', ('time_counter',), zlib=True);
time_counter.units = 's';
time_counter.longname = 'time';

vozocrtx = bdy_file.createVariable('vozocrtx', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);
vomecrty = bdy_file.createVariable('vomecrty', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);
votemper = bdy_file.createVariable('votemper', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);
vosaline = bdy_file.createVariable('vosaline', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);

vozocrtx[:] = u_3d_bottom[:];
vomecrty[:] = v_3d_bottom[:];
votemper[:] = votemper_bottom[:];
vosaline[:] = vosaline_bottom[:];

bdy_file.close()

In [30]:
u_3d_bottom.shape

(43, 50, 4, 70)

In [31]:
v_3d_bottom.shape

(43, 50, 4, 70)

In [32]:
votemper_bottom.shape

(43, 50, 4, 70)